In [2]:
import os, glob
from os.path import abspath, join
import re
from numpy import array
import pandas as pd

In [3]:
def parse_out(fout):
    from numpy import array
    regex = re.compile('uniprotkb:P([0-9]*)')
    get_score = re.compile(r"{u'main': ([+-]?\d+\.\d+|\d+)")
    get_params = re.compile(r"}]\n(.*)\n")                   

    score = get_score.findall(fout)[0]
    params = get_params.findall(fout)[0]                   
    return {'score':float(score), 'params':params}

def extract_output(exp_dir):
    outfiles = glob.glob(join(exp_dir, 'output/*.out'))
    experout = []
    for outfile in outfiles:
        outname = os.path.basename(outfile)
        with open(outfile) as f:
            fout = f.read()
            try:
                experout.append(parse_out(fout))
            except IndexError:
                pass
            # add output file name
            experout[-1].update({'outfile':outname})
    return experout

In [4]:
dirs_smint = [abspath(d) for d in os.listdir('.') if 'smint' in d and os.path.isdir(d)]
expout = {}
for path_out in dirs_smint:
    print 'exctracting output from:', path_out
    experiment = os.path.basename(path_out)
    expout[experiment] = extract_output(path_out)

exctracting output from: /backup/portland/smint_veh_1m
exctracting output from: /backup/portland/smint_veh_2m
exctracting output from: /backup/portland/smint_bur_3m
exctracting output from: /backup/portland/smint_bur_2m
exctracting output from: /backup/portland/smint_bur_1m
exctracting output from: /backup/portland/smint_veh_3m


In [5]:
best = []
for experiment, output in expout.iteritems():
    print 'Choosing best trial in experiment:', experiment
    # select best score
    df = pd.DataFrame(expout[experiment])
    best_out = df.iloc[df.score.argmin()].to_dict()
    best_out.update({'experiment':experiment})
    best.append(best_out)

Choosing best trial in experiment: smint_bur_2m
Choosing best trial in experiment: smint_bur_3m
Choosing best trial in experiment: smint_bur_1m
Choosing best trial in experiment: smint_veh_1m
Choosing best trial in experiment: smint_veh_2m
Choosing best trial in experiment: smint_veh_3m


In [6]:
b = pd.DataFrame(best).set_index('experiment')

In [10]:
b['params'].values

array([ "{u'k': array([50]), u'kde_lags': array([1]), u'lt': array([ 0.5]), u'kde_bw': array([250]), u'eta': array([ 0.5]), u'kde_win': array([3]), u'theta': array([ 0.]), u'dely': array([ 250.]), u'delx': array([ 250.])}",
       'Anything printed here will end up in the output directory for job #7',
       "{u'k': array([186]), u'kde_lags': array([4]), u'lt': array([ 0.5]), u'kde_bw': array([250]), u'eta': array([ 1.]), u'kde_win': array([30]), u'theta': array([ 0.]), u'dely': array([ 252.89219474]), u'delx': array([ 335.8964064])}",
       "{u'k': array([84]), u'kde_lags': array([1]), u'lt': array([ 0.5]), u'kde_bw': array([250]), u'eta': array([ 1.]), u'kde_win': array([22]), u'theta': array([ 0.]), u'dely': array([ 260.55790904]), u'delx': array([ 250.])}",
       "{u'k': array([54]), u'kde_lags': array([1]), u'lt': array([ 0.50612386]), u'kde_bw': array([500]), u'eta': array([ 0.74804254]), u'kde_win': array([4]), u'theta': array([ 0.20819929]), u'dely': array([ 250.39414133]), u